# Setting up XTDB to use Postgres with Egeria Workspaces
## Pre-reqs 
* A running Egeria Workspaces environment. You can either use one of the sample docker compose scripts that starts up postgres along with egeria or you can use an external postgres.


## Background

Egeria supports several repository technologies including XTDB. XTDB itself can be configured to use different backend storage mechanisms including Key-Value (KV) stores, Kafka and databases such as Postgres. XTDB is typically configured to use multiple technologies for different purposes. XTDB capabilities can use different technologies for:

* index - store 
* search
* tx-log
* document-store

For development, our default configuration uses XTDB with the KV stores for the index-store, tx-log and document-store. However, sometimes it is useful to change the configuration to Postgres for the tx-log and document-store, in particular, for testing. Many other configurations are possible.

This workbook describes how to switch the repository for the **qs-metadata-store** from the default to Postgres.

## Approach

To switch the repository of an Egeria **Metadata Access Store** server, we will need to update the server's configuration document with the new connection information and then restart it. This notebook provides a guide on how to do this. While this approach will work for most Egeria deployment environments, to be more specific we will focus on environments deploying Egeria Workspaces.

The **Egeria Workspaces** github repository contains several different **Docker Compose** scripts that configure different combinations of technologies for use in development and experimentation of Egeria environments. We will use one of these configurations, called **egeria-platform-jupyter-proxy-pg-compose** which, as the name suggests, deploys an environment that includes Egeria, Jupyter Server, the Open Lineage Proxy Backend and Postgres. With a minor configuration change we will have Egeria use Postgres with XTDB for the **qs-metadata-store** server repository. 

## Steps
1) Install docker with docker compose and start it (or a compatible alternate technology)
2) Download a clone of egeria-workspaces and start the Egeria environment by:
    - `git clone https://github.com/odpi/egeria-workspaces`
    - in a terminal window, cd to the `egeria-workspaces\compose-configs/egeria-platform-jupyter-proxy-pg-compose` directory
    - enter the command `docker compose -f egeria-platform-jupyter-proxy-pg-compose up  --build`
    This command should startup a number of docker containers - it may take a several minutes for everything to be ready.
3) Run this **Jupyter Notebook** 
    - by opening jupyter in a browser - open the URL `https://localhost:8888`and type Egeria into the password box.
    - This notebook is kept in the director `workbooks/optional configurations/XTDB`
    - Continue to follow the guidance in this notebook by executing each cell.

## Import the required python libraries and set up the environment

In [ ]:
import os
from pyegeria import EgeriaTech, CoreServerConfig

In [ ]:
import asyncio
import nest_asyncio
nest_asyncio.apply()


## We will be updating the **metadata_store**

In [ ]:
view_server = os.environ.get("VIEW_SERVER","view-server")
url = os.environ.get("EGERIA_VIEW_SERVER_URL","https://host.docker.internal:9443")
user_id = os.environ.get("EGERIA_USER", "garygeeke")
user_pwd = os.environ.get("EGERIA_USER_PASSWORD")
metadata_store = os.environ.get("EGERIA_METADATA_STORE")


## Set up the PostgreSQL configuration
The configuration below works for the PostgreSQL server deployed in this docker configuraiton. You can see the PostgreSQL container. If you wanted to use a different postgres deployment, you could change these parameters.

In [ ]:
pg_user = 'postgres'
pg_pwd = 'egeria'
pg_host = "host.docker.internal"
pg_port = "5442"

## Now we will update the configuration of the **metadata_store**

In [ ]:
o_client: CoreServerConfig = CoreServerConfig(metadata_store, url, user_id, user_pwd)
o_client.set_xtdb_pg_repository(pg_host, pg_port, pg_user, pg_pwd)


## Finally, we restart the server so that it picks up its new configuration.
* Don't be concerned about a timeout error if it occurs - in some environments, it takes longer for the **CoreContentPack** to be loaded. 
* Its a good idea to look at the console log to check for any errors - if you are running Egeria in docker, one way to check is by looking at the logs for the egeria-main-1 container.

In [ ]:
r_client = EgeriaTech(view_server, url, 'garygeeke', user_pwd)
token = r_client.create_egeria_bearer_token()
r_client.activate_server_with_stored_config(
                None, metadata_store
            )

In [ ]:
help (EgeriaTech.activate_server_with_stored_config)